<a href="https://colab.research.google.com/github/DeJezuz/AI-Future-Directions-Assignment/blob/main/model_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# AI Future Directions Assignment - Edge AI Prototype

## 1. Setup and Data Loading
# Import necessary libraries: TensorFlow, TensorFlow Lite, NumPy, Matplotlib
import tensorflow as tf
from tensorflow import keras
import numpy as np

# [Code to load and preprocess your dataset, e.g., 'Trashnet' or 'RecycleNet']
# For quantization, you will need a representative dataset. This is a placeholder.
# Replace this with actual representative samples from your training/validation data.
# For example, if your input images are 224x224x3 and normalized to [0, 1] floats:
def representative_dataset_gen():
    # In a real scenario, this would load a small subset of your training data
    # and preprocess it exactly as your model expects.
    # For demonstration, we yield random data.
    for _ in range(100): # Yield 100 samples
        # Model input shape (224, 224, 3), type tf.float32 (before quantization)
        yield [np.random.uniform(0, 1, size=(1, 224, 224, 3)).astype(np.float32)]

# [Code to split data into training, validation, and test sets]

## 2. Model Training (Use a Lightweight Architecture)
# Use MobileNetV2 or a similar model for transfer learning
base_model = keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model's layers
base_model.trainable = False

# Add your classification layers (GlobalAveragePooling2D, Dense) here
model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1, activation='sigmoid') # Assuming binary classification (e.g., trash vs. recycle)
])

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

# [Train the model: model.fit(...)]

# Evaluate model before conversion
# print("Classical Model Accuracy:", model.evaluate(test_data)) # test_data is not defined yet

## 3. Model Conversion to TensorFlow Lite
# Define the path to save the converted model
TFLITE_MODEL_FILE = 'model.tflite'

# Instantiate the TFLite Converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply Optimization (Quantization) for Edge deployment
# For best performance on edge devices, use 8-bit integer quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.uint8 # Or tf.float32 if not fully quantizing
converter.inference_output_type = tf.uint8

# Specify the representative dataset for full integer quantization
converter.representative_dataset = representative_dataset_gen

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model file
with open(TFLITE_MODEL_FILE, 'wb') as f:
    f.write(tflite_model)

print(f"\nModel converted and saved as {TFLITE_MODEL_FILE}")

## 4. TFLite Model Testing (Verification of Accuracy and Speed)
# [Code to load the TFLite model using tf.lite.Interpreter]
# [Code to run inference on a sample of the test data]
# [Code to calculate TFLite model accuracy and measure inference time (speed)]

Saved artifact at '/tmp/tmp2frwtttq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_940')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132668624290256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668624304848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668624305232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668624305808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668624304272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668624290064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668624304656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668624304464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668624306000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668624303312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1326686243

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(



Model converted and saved as model.tflite
